In [1]:
#Import relevant libraries
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
from scipy import ndimage
from tvtk.api import tvtk
from mayavi import mlab
import math

In [5]:
#da and db = diffusion rates, dt = time step, t = starting time
da,db,dt,t = .24,.12,1,0
width = 32
stoptime = 1000
f = .0367
k = .0649


#3x3 kernel for convolution
        
kernel = np.array([[[1/26, 1/26, 1/26],[1/26, 1/26, 1/26], [1/26, 1/26, 1/26]],
                 [[1/26, 1/26, 1/26],[1/26, -1, 1/26], [1/26, 1/26, 1/26]],
                 [[1/26, 1/26, 1/26],[1/26, 1/26, 1/26], [1/26, 1/26, 1/26]]])

#initialise the grid, A=1,  B=0, and C=0
A = np.ones([width,width,width])
B = np.zeros([width,width,width])
C = np.zeros([width,width,width])

#add small seed area
x = int(width/2)
B[x:x+1+int(math.ceil(width/64)),x:x+1+int(math.ceil(width/64)),x:x+1+int(math.ceil(width/64))] = 1

#update the grid using the diffusion equations
while t<stoptime:      
    A += (da*ndimage.convolve(A,kernel,mode='reflect',cval=0.0) - A*B**2 + f*(1-A))*dt
    B += (db*ndimage.convolve(B,kernel,mode='reflect',cval=0.0) + A*B**2 - (k+f)*B)*dt
    t +=  dt
    
    


In [4]:
data = B
mlab.contour3d(data)
mlab.show()
